In [17]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
from pudl import ferc1, pudl, models, models_ferc1, settings, constants
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.models
%aimport pudl.models_ferc1
%aimport pudl.eia923
%aimport pudl.constants

In [3]:
ferc1_engine = pudl.ferc1.db_connect_ferc1()
pudl.ferc1.init_db(refyear=2015, years=range(2004,2016), def_db=True)

Defining new FERC Form 1 DB based on 2015...
Clearing any existing FERC Form 1 database MetaData...
Ingesting FERC Form 1 Data from 2004...
Ingesting FERC Form 1 Data from 2005...
Ingesting FERC Form 1 Data from 2006...
Ingesting FERC Form 1 Data from 2007...
Ingesting FERC Form 1 Data from 2008...
Ingesting FERC Form 1 Data from 2009...
Ingesting FERC Form 1 Data from 2010...
Ingesting FERC Form 1 Data from 2011...
Ingesting FERC Form 1 Data from 2012...
Ingesting FERC Form 1 Data from 2013...
Ingesting FERC Form 1 Data from 2014...
Ingesting FERC Form 1 Data from 2015...


In [4]:
ferc1_respondents = pd.read_sql('''SELECT respondent_id, respondent_name FROM f1_respondent_id''', ferc1_engine)
ferc1_steam_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, tot_capacity
                                    FROM f1_steam''', ferc1_engine)
ferc1_small_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, kind_of_fuel, capacity_rating
                                    FROM f1_gnrt_plant''', ferc1_engine)
ferc1_hydro_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_hydro''', ferc1_engine)
ferc1_pumped_storage_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_pumped_storage''', ferc1_engine)

In [5]:
ferc1_all_plants = pd.concat([ferc1_steam_plants, ferc1_small_plants, ferc1_hydro_plants, ferc1_pumped_storage_plants])
ferc1_all_plants = ferc1_respondents.merge(ferc1_all_plants, how='left', on='respondent_id')
ferc1_all_plants = ferc1_all_plants.replace('^$', np.nan, regex=True)
ferc1_all_plants = ferc1_all_plants.dropna(subset=['plant_name',])
ferc1_all_plants['report_year'] = ferc1_all_plants.report_year.astype(int)
ordered_for_cgosnell = ['respondent_id', 'respondent_name', 'plant_name', 'report_year',\
                        'kind_of_fuel',  'capacity_rating', 'plant_kind', 'tot_capacity']
ferc1_all_plants = ferc1_all_plants[ordered_for_cgosnell]
ferc1_all_plants = ferc1_all_plants.drop_duplicates(subset=['respondent_id','plant_name'])

In [6]:
ferc1_all_plants.to_csv('../../results/id_mapping/ferc1_all_plants_2004-2015.csv', index=False)

In [7]:
ferc1_respondents.to_csv('../../results/id_mapping/ferc1_all_respondents_2004-2015.csv', index=False)

In [8]:
ferc1_plants_all_years = pd.concat([ferc1_steam_plants, ferc1_small_plants, ferc1_hydro_plants, ferc1_pumped_storage_plants])
ferc1_plants_all_years = ferc1_respondents.merge(ferc1_plants_all_years, how='left', on='respondent_id')
ferc1_plants_all_years = ferc1_plants_all_years.replace('^$', np.nan, regex=True)
ferc1_plants_all_years = ferc1_plants_all_years.dropna(subset=['plant_name',])
ferc1_plants_all_years['report_year'] = ferc1_plants_all_years.report_year.astype(int)
ordered_for_cgosnell = ['respondent_id', 'respondent_name', 'plant_name','report_year','capacity_rating','tot_capacity']
                        #'kind_of_fuel',  'capacity_rating', 'plant_kind', 'tot_capacity']
ferc1_plants_all_years = ferc1_plants_all_years[ordered_for_cgosnell]
ferc1_plants_all_years['plant_name'] = ferc1_plants_all_years.plant_name.str.strip().str.lower()
ferc1_plants_all_years['plant_name'] = ferc1_plants_all_years.plant_name.str.strip().str.title()
ferc1_plants_all_years = ferc1_plants_all_years[(ferc1_plants_all_years['tot_capacity']>5) |
    (ferc1_plants_all_years['capacity_rating']>5)]
ferc1_plants_all = ferc1_plants_all_years.drop_duplicates(subset=['respondent_id','plant_name'])
ferc1_plants_all = ferc1_plants_all[['respondent_id', 'respondent_name', 'plant_name']]
ferc1_plants_all.count()

respondent_id      2101
respondent_name    2101
plant_name         2101
dtype: int64

In [9]:
ferc1_plants_2015 = ferc1_plants_all_years[ferc1_plants_all_years.report_year == 2015]
ferc1_plants_2015 = ferc1_plants_2015.drop_duplicates(subset=['respondent_id','plant_name'])
ferc1_plants_2015 = ferc1_plants_2015[['respondent_id', 'respondent_name', 'plant_name']]
ferc1_plants_2015.count()

respondent_id      1350
respondent_name    1350
plant_name         1350
dtype: int64

In [10]:
ferc1_plants_2015_tuple = set([ tuple(line) for line in ferc1_plants_2015.values.tolist()])
ferc1_plants_all_tuple = set([ tuple(line) for line in ferc1_plants_all.values.tolist()])
ferc1_plants_other_years = pd.DataFrame(list(ferc1_plants_all_tuple.difference(ferc1_plants_2015_tuple)))
ferc1_plants_other_years.count()

0    751
1    751
2    751
dtype: int64

In [11]:
ferc1_plants_2015_tuple

{(2, 'ALABAMA POWER COMPANY', 'Barry Cc'),
 (6, 'Appalachian Power Company', 'Dresden'),
 (70, 'Idaho Power Company', 'Swan Falls'),
 (186, 'VIRGINIA ELECTRIC AND POWER COMPANY', 'Yorktown'),
 (210,
  'MidAmerican Energy Company',
  'Vienna Wind Farm (64 Units @ 2.346 Mw Each)'),
 (55, 'Florida Power Corporation', 'Avon Park'),
 (122, 'NorthWestern Corporation', 'Kerr'),
 (57, 'Georgia Power Company', 'Sinclair Dam'),
 (159, 'South Carolina Electric & Gas Company', 'Fairfield'),
 (45, 'Duke Energy Carolinas, LLC', 'Buck'),
 (95, 'MDU Resources Group, Inc.', 'Thunder Spirit'),
 (120, 'Northern States Power Company (Minnesota)', 'Black Dog 3 & 4'),
 (132, 'Otter Tail Corporation', 'Jamestown'),
 (44, 'The Detroit Edison Company', 'Monroe'),
 (82, 'Kentucky Utilities Company', 'Dix Dam'),
 (122, 'NorthWestern Corporation', 'Madison'),
 (11, 'BANGOR HYDRO-ELECTRIC COMPANY', 'Total'),
 (159, 'South Carolina Electric & Gas Company', 'Hagood #4'),
 (85, 'KeySpan Generation, LLC', 'East Hampto

In [12]:
ferc1_plants_all.to_csv('../../results/id_mapping/ferc1_all_plants_2004-2015.csv', index=False)
ferc1_respondents.to_csv('../../results/id_mapping/ferc1_all_respondents_2004-2015.csv', index=False)